## YOLO3

In [13]:
"""
Run a YOLO_v3 style detection model on test images.
"""

import colorsys
import os
import random
from timeit import time
from timeit import default_timer as timer  ### to calculate FPS

import numpy as np
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval
from yolo3.utils import letterbox_image

class YOLO(object):
    def __init__(self):
        self.model_path = 'model_data/yolo.h5'
        self.anchors_path = 'model_data/yolo_anchors.txt'
        self.classes_path = 'model_data/coco_classes.txt'
        self.score = 0.3
        self.iou = 0.5
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416) # fixed size or (None, None)
        self.is_fixed_size = self.model_image_size != (None, None)
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
            anchors = [float(x) for x in anchors.split(',')]
            anchors = np.array(anchors).reshape(-1, 2)
        return anchors

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'

        self.yolo_model = load_model(model_path, compile=False)
        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        #random.seed(10101)  # Fixed seed for consistent colors across runs.
        random.seed(229)  # Fixed seed for consistent colors across runs.
        random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = time.time()
        count = 0

        if self.is_fixed_size:
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        #print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        #print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            #label = '{} {:.2f}'.format(predicted_class, score)
            label = '{}, {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            
            count += 1
            #print(label, (left, top), (right, bottom))
            #print("{}, {}, {}, {}, {}".format(label, left, top, right, bottom))
            #print("{}, {}, {}, {}, {}, {}".format(count - 1, label, left, top, right, bottom))
            print("{}, {}, {}, {}, {}, {}".format(len(out_boxes), label, left, top, right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        end = time.time()
        #print(end - start)
        return image

    def close_session(self):
        self.sess.close()

## Detect Video

In [14]:
def detect_video(yolo, video_path):
    import cv2
    vid = cv2.VideoCapture(video_path)
    
    #print("Video Dim: {}  {}  {}".format(vid.get(3), 'X', vid.get(4)))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (1920,1080))
    out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (960,540))      # resize image half
    
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    #start4Total = timer()
    cnt = 0
    while True:
        return_value, frame = vid.read()
        
        if return_value == True:
        
            cnt += 1
            # Re-size
            #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
            # Re-size
            image = Image.fromarray(frame)
            image = yolo.detect_image(image)
            result = np.asarray(image)
            curr_time = timer()
            exec_time = curr_time - prev_time
            prev_time = curr_time
            accum_time = accum_time + exec_time
            curr_fps = curr_fps + 1
            if accum_time > 1:
                accum_time = accum_time - 1
                fps = "FPS: " + str(curr_fps)
                curr_fps = 0
            
            print(cnt-1)
            
            cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
            cv2.namedWindow("result", cv2.WINDOW_NORMAL)
            out.write(result)
            cv2.imshow("result", result)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
            
    #print('++++++++ Video End ++++++++')
    
    vid.release()
    out.release()
    cv2.destroyAllWindows()
    
    #print('\nTotal elapsed time = ' + str(timer() - start4Total) + ' s\n')

    #yolo.close_session()

## Downsample

In [16]:
def detect_video(yolo, video_path):
    import cv2
    vid = cv2.VideoCapture(video_path)
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (1920,1080))
    out = cv2.VideoWriter('test_python_yolo3.avi',fourcc, 30.0, (960,540))      # resize image half
    i = 0
    frame_rate_divider = 3
    
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    #start4Total = timer()
    cnt = 0
    while True:
        return_value, frame = vid.read()
        
        if return_value == True:
            if i % frame_rate_divider == 0:
                
                cnt += 1
                # Re-size
                #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
                # Re-size
                image = Image.fromarray(frame)
                image = yolo.detect_image(image)
                result = np.asarray(image)
                curr_time = timer()
                exec_time = curr_time - prev_time
                prev_time = curr_time
                accum_time = accum_time + exec_time
                curr_fps = curr_fps + 1
                if accum_time > 1:
                    accum_time = accum_time - 1
                    fps = "FPS: " + str(curr_fps)
                    curr_fps = 0
            
                print(cnt-1)
            
                cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
                cv2.namedWindow("result", cv2.WINDOW_NORMAL)
                out.write(result)
                cv2.imshow("result", result)
                i += 1
            else:
                i += 1
                
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
            
    #print('++++++++ Video End ++++++++')
    
    vid.release()
    out.release()
    cv2.destroyAllWindows()
    
    #print('\nTotal elapsed time = ' + str(timer() - start4Total) + ' s\n')

    #yolo.close_session()

## Detect Video

In [17]:
#from yolo import YOLO
#from yolo import detect_video

if __name__ == '__main__':
    #video_path='/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4'
    video_path='/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/peoplecars.mp4'
    detect_video(YOLO(), video_path)


model_data/yolo.h5 model, anchors, and classes loaded.
7, car, 3, 0.37, 216, 496, 482, 540
7, car, 3, 0.90, 419, 239, 562, 334
7, car, 3, 0.96, 712, 368, 944, 487
7, car, 3, 0.99, 788, 207, 953, 281
7, person, 1, 0.47, 44, 200, 71, 258
7, person, 1, 0.52, 20, 197, 46, 264
7, person, 1, 0.91, 31, 198, 64, 262
0
6, car, 3, 0.41, 216, 496, 482, 540
6, car, 3, 0.90, 419, 239, 561, 335
6, car, 3, 0.96, 712, 369, 944, 487
6, car, 3, 0.99, 788, 209, 953, 281
6, person, 1, 0.47, 49, 200, 71, 258
6, person, 1, 0.97, 31, 198, 60, 264
1
7, truck, 8, 0.30, 688, 362, 931, 489
7, car, 3, 0.43, 217, 495, 481, 540
7, car, 3, 0.89, 419, 239, 562, 336
7, car, 3, 0.95, 712, 368, 943, 487
7, car, 3, 0.99, 789, 209, 953, 280
7, person, 1, 0.90, 31, 199, 68, 263
7, person, 1, 0.91, 46, 200, 82, 260
2
5, car, 3, 0.46, 218, 495, 481, 540
5, car, 3, 0.92, 419, 239, 562, 335
5, car, 3, 0.95, 713, 369, 943, 487
5, car, 3, 0.99, 789, 209, 953, 281
5, person, 1, 0.89, 42, 199, 86, 263
3
6, car, 3, 0.41, 217, 496, 

11, truck, 8, 0.33, 688, 363, 931, 489
11, car, 3, 0.36, 216, 497, 481, 540
11, car, 3, 0.94, 419, 239, 561, 335
11, car, 3, 0.95, 711, 369, 944, 487
11, car, 3, 0.99, 788, 208, 952, 280
11, person, 1, 0.77, 0, 159, 10, 248
11, person, 1, 0.80, 27, 182, 52, 243
11, person, 1, 0.88, 53, 180, 79, 246
11, person, 1, 0.97, 182, 208, 214, 262
11, person, 1, 0.99, 154, 205, 186, 271
11, person, 1, 0.99, 82, 187, 121, 258
26
11, truck, 8, 0.33, 689, 362, 931, 488
11, car, 3, 0.32, 215, 497, 483, 540
11, car, 3, 0.93, 418, 238, 561, 335
11, car, 3, 0.95, 711, 368, 944, 487
11, car, 3, 0.99, 787, 208, 952, 281
11, person, 1, 0.79, 57, 182, 84, 244
11, person, 1, 0.82, 187, 206, 214, 263
11, person, 1, 0.91, 0, 169, 14, 244
11, person, 1, 0.95, 32, 183, 56, 243
11, person, 1, 0.99, 86, 188, 123, 258
11, person, 1, 0.99, 158, 203, 191, 272
27
11, truck, 8, 0.35, 689, 362, 931, 488
11, car, 3, 0.30, 215, 498, 482, 540
11, car, 3, 0.94, 418, 238, 561, 335
11, car, 3, 0.95, 710, 369, 945, 486
11, ca

10, truck, 8, 0.35, 689, 362, 930, 489
10, car, 3, 0.95, 711, 369, 944, 487
10, car, 3, 0.98, 420, 239, 559, 335
10, car, 3, 0.99, 786, 209, 951, 281
10, person, 1, 0.65, 139, 176, 163, 239
10, person, 1, 0.94, 299, 211, 320, 273
10, person, 1, 0.97, 82, 176, 110, 249
10, person, 1, 0.99, 158, 177, 184, 244
10, person, 1, 0.99, 266, 209, 296, 280
10, person, 1, 0.99, 191, 186, 229, 257
46
10, truck, 8, 0.33, 688, 362, 931, 489
10, car, 3, 0.95, 711, 369, 944, 487
10, car, 3, 0.98, 420, 239, 559, 335
10, car, 3, 0.99, 786, 208, 951, 281
10, person, 1, 0.85, 304, 212, 326, 273
10, person, 1, 0.88, 138, 178, 167, 240
10, person, 1, 0.98, 160, 178, 191, 243
10, person, 1, 0.98, 85, 178, 116, 247
10, person, 1, 0.99, 271, 210, 301, 280
10, person, 1, 1.00, 194, 187, 232, 260
47
10, truck, 8, 0.34, 688, 362, 931, 489
10, car, 3, 0.95, 712, 369, 944, 487
10, car, 3, 0.98, 420, 239, 559, 335
10, car, 3, 0.99, 786, 208, 951, 281
10, person, 1, 0.97, 142, 178, 173, 239
10, person, 1, 0.98, 91, 1

12, car, 3, 0.34, 224, 496, 479, 540
12, car, 3, 0.96, 712, 369, 944, 487
12, car, 3, 0.97, 423, 240, 561, 338
12, car, 3, 1.00, 789, 209, 952, 280
12, person, 1, 0.95, 6, 184, 35, 250
12, person, 1, 0.96, 437, 223, 460, 254
12, person, 1, 0.96, 397, 218, 430, 290
12, person, 1, 0.97, 107, 195, 144, 251
12, person, 1, 0.97, 255, 173, 283, 234
12, person, 1, 0.98, 334, 189, 363, 260
12, person, 1, 0.98, 206, 185, 231, 251
12, person, 1, 0.99, 287, 173, 319, 233
70
13, car, 3, 0.96, 711, 369, 945, 487
13, car, 3, 0.98, 423, 240, 561, 338
13, car, 3, 1.00, 789, 210, 952, 280
13, person, 1, 0.32, 0, 198, 8, 249
13, person, 1, 0.39, 1, 184, 19, 258
13, person, 1, 0.95, 441, 225, 463, 252
13, person, 1, 0.96, 11, 184, 35, 249
13, person, 1, 0.96, 405, 217, 433, 281
13, person, 1, 0.97, 263, 173, 286, 236
13, person, 1, 0.97, 113, 194, 155, 250
13, person, 1, 0.99, 289, 173, 322, 235
13, person, 1, 0.99, 338, 189, 372, 261
13, person, 1, 0.99, 207, 185, 244, 252
71
15, skateboard, 37, 0.32, 1

13, car, 3, 0.35, 218, 496, 479, 540
13, car, 3, 0.96, 711, 370, 942, 487
13, car, 3, 0.99, 421, 244, 557, 338
13, car, 3, 1.00, 785, 211, 951, 280
13, person, 1, 0.65, 501, 237, 527, 262
13, person, 1, 0.74, 58, 188, 83, 251
13, person, 1, 0.86, 458, 223, 489, 248
13, person, 1, 0.88, 71, 186, 101, 249
13, person, 1, 0.90, 290, 187, 317, 248
13, person, 1, 0.94, 318, 170, 339, 230
13, person, 1, 0.99, 273, 189, 300, 251
13, person, 1, 0.99, 357, 171, 387, 228
13, person, 1, 0.99, 415, 191, 445, 258
85
15, car, 3, 0.38, 217, 495, 479, 540
15, car, 3, 0.96, 711, 370, 943, 487
15, car, 3, 0.99, 421, 245, 558, 339
15, car, 3, 1.00, 786, 211, 951, 280
15, person, 1, 0.40, 299, 187, 317, 229
15, person, 1, 0.44, 502, 237, 528, 263
15, person, 1, 0.74, 507, 234, 533, 274
15, person, 1, 0.76, 459, 221, 492, 249
15, person, 1, 0.83, 291, 186, 317, 249
15, person, 1, 0.93, 84, 184, 109, 250
15, person, 1, 0.94, 67, 189, 93, 253
15, person, 1, 0.97, 276, 188, 302, 251
15, person, 1, 0.97, 318, 1

...

15, skateboard, 37, 0.64, 840, 290, 887, 307
15, car, 3, 0.37, 799, 213, 950, 284
15, car, 3, 0.40, 219, 493, 482, 540
15, car, 3, 0.83, 425, 239, 561, 335
15, car, 3, 0.95, 710, 367, 949, 487
15, person, 1, 0.76, 809, 168, 835, 208
15, person, 1, 0.76, 533, 180, 561, 239
15, person, 1, 0.78, 865, 227, 896, 303
15, person, 1, 0.81, 533, 224, 560, 293
15, person, 1, 0.81, 878, 222, 903, 300
15, person, 1, 0.94, 390, 221, 412, 290
15, person, 1, 0.98, 784, 206, 820, 286
15, person, 1, 0.98, 698, 176, 728, 237
15, person, 1, 0.99, 572, 167, 600, 230
15, person, 1, 0.99, 654, 189, 686, 268
283
13, car, 3, 0.37, 217, 494, 483, 540
13, car, 3, 0.59, 784, 212, 955, 283
13, car, 3, 0.88, 425, 239, 561, 335
13, car, 3, 0.97, 711, 367, 947, 486
13, person, 1, 0.38, 536, 180, 564, 237
13, person, 1, 0.75, 397, 222, 419, 287
13, person, 1, 0.77, 805, 162, 836, 214
13, person, 1, 0.89, 536, 183, 567, 297
13, person, 1, 0.95, 578, 168, 600, 227
13, person, 1, 0.96, 785, 207, 822, 289
13, person, 1, 

11, car, 3, 0.97, 424, 252, 562, 339
11, car, 3, 0.98, 712, 375, 948, 491
11, car, 3, 0.99, 799, 214, 945, 285
11, person, 1, 0.56, 763, 181, 782, 222
11, person, 1, 0.58, 600, 193, 624, 256
11, person, 1, 0.71, 463, 231, 481, 251
11, person, 1, 0.83, 599, 212, 625, 310
11, person, 1, 0.92, 634, 173, 655, 235
11, person, 1, 0.94, 754, 191, 786, 298
11, person, 1, 0.95, 812, 167, 840, 218
11, person, 1, 0.96, 856, 221, 885, 299
299
11, car, 3, 0.97, 423, 250, 563, 337
11, car, 3, 0.97, 713, 373, 947, 489
11, car, 3, 0.99, 802, 214, 945, 283
11, person, 1, 0.37, 496, 275, 519, 292
11, person, 1, 0.70, 467, 229, 484, 249
11, person, 1, 0.87, 601, 191, 628, 256
11, person, 1, 0.88, 638, 171, 659, 233
11, person, 1, 0.91, 602, 240, 630, 312
11, person, 1, 0.95, 812, 165, 840, 217
11, person, 1, 0.98, 757, 190, 789, 298
11, person, 1, 0.99, 858, 220, 891, 300
300
12, truck, 8, 0.35, 680, 365, 937, 497
12, car, 3, 0.94, 804, 213, 942, 285
12, car, 3, 0.97, 713, 371, 944, 489
12, car, 3, 0.97,

12, car, 3, 0.96, 426, 247, 558, 337
12, car, 3, 0.98, 710, 374, 946, 489
12, car, 3, 0.99, 795, 211, 947, 285
12, person, 1, 0.39, 536, 241, 553, 278
12, person, 1, 0.54, 497, 275, 516, 291
12, person, 1, 0.91, 532, 231, 557, 296
12, person, 1, 0.93, 851, 169, 874, 207
12, person, 1, 0.94, 940, 221, 959, 303
12, person, 1, 0.94, 671, 191, 700, 261
12, person, 1, 0.98, 704, 174, 727, 236
12, person, 1, 0.99, 877, 233, 908, 314
12, person, 1, 0.99, 652, 259, 690, 329
319
13, car, 3, 0.93, 426, 245, 559, 337
13, car, 3, 0.98, 711, 373, 947, 489
13, car, 3, 0.98, 794, 211, 950, 287
13, person, 1, 0.34, 541, 241, 557, 279
13, person, 1, 0.52, 849, 162, 880, 212
13, person, 1, 0.62, 496, 274, 516, 290
13, person, 1, 0.79, 853, 171, 875, 204
13, person, 1, 0.87, 944, 210, 959, 310
13, person, 1, 0.87, 676, 191, 702, 263
13, person, 1, 0.90, 539, 233, 560, 297
13, person, 1, 0.94, 878, 232, 921, 316
13, person, 1, 0.99, 705, 174, 730, 236
13, person, 1, 0.99, 656, 256, 694, 328
320
11, car, 3

10, car, 3, 0.97, 711, 372, 947, 488
10, car, 3, 0.97, 424, 243, 560, 337
10, car, 3, 1.00, 790, 208, 952, 288
10, person, 1, 0.30, 917, 170, 934, 214
10, person, 1, 0.33, 944, 184, 960, 263
10, person, 1, 0.75, 914, 168, 937, 232
10, person, 1, 0.92, 783, 175, 808, 240
10, person, 1, 0.97, 638, 241, 667, 311
10, person, 1, 0.98, 748, 201, 775, 271
10, person, 1, 0.99, 755, 271, 802, 347
343
10, car, 3, 0.97, 425, 243, 560, 336
10, car, 3, 0.97, 712, 372, 947, 488
10, car, 3, 1.00, 789, 207, 951, 287
10, person, 1, 0.37, 946, 181, 960, 268
10, person, 1, 0.53, 789, 183, 811, 218
10, person, 1, 0.69, 644, 241, 670, 308
10, person, 1, 0.91, 915, 169, 937, 230
10, person, 1, 0.91, 785, 177, 814, 236
10, person, 1, 0.99, 746, 203, 780, 273
10, person, 1, 0.99, 757, 271, 800, 349
344
8, car, 3, 0.96, 700, 368, 952, 502
8, car, 3, 0.97, 425, 243, 560, 336
8, car, 3, 1.00, 788, 206, 950, 288
8, person, 1, 0.81, 788, 177, 817, 234
8, person, 1, 0.86, 917, 168, 939, 229
8, person, 1, 0.88, 650,

6, car, 3, 0.78, 798, 202, 944, 284
6, car, 3, 0.96, 423, 243, 562, 335
6, car, 3, 0.97, 707, 369, 949, 503
6, person, 1, 0.72, 885, 176, 906, 221
6, person, 1, 0.92, 772, 247, 802, 322
6, person, 1, 0.99, 875, 274, 923, 348
371
8, car, 3, 0.37, 873, 230, 957, 285
8, car, 3, 0.82, 798, 203, 943, 283
8, car, 3, 0.95, 423, 242, 562, 335
8, car, 3, 0.98, 707, 368, 949, 504
8, person, 1, 0.56, 842, 209, 866, 289
8, person, 1, 0.69, 775, 246, 807, 324
8, person, 1, 0.74, 892, 175, 912, 226
8, person, 1, 0.99, 876, 273, 927, 352
372
7, car, 3, 0.89, 799, 204, 945, 284
7, car, 3, 0.95, 423, 242, 562, 335
7, car, 3, 0.97, 706, 368, 950, 502
7, person, 1, 0.55, 894, 177, 914, 227
7, person, 1, 0.62, 842, 209, 867, 289
7, person, 1, 0.97, 876, 272, 928, 353
7, person, 1, 0.98, 780, 258, 819, 324
373
7, car, 3, 0.88, 797, 204, 943, 285
7, car, 3, 0.96, 423, 242, 562, 335
7, car, 3, 0.97, 706, 366, 949, 502
7, person, 1, 0.50, 842, 209, 867, 290
7, person, 1, 0.61, 898, 181, 919, 231
7, person, 1,